In [ ]:
!pip install langchain_openai
!pip install langchain-community
!pip install pypdf
!pip install faiss-cpu

# 1. 사용 환경 준비

In [ ]:
import os
from getpass import getpass

os.environ["OPENAI_API_KEY"] = getpass("OpenAI API key 입력: ")

OpenAI API key 입력: ··········


# 2. 모델 초기화

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

model = ChatOpenAI(model="gpt-4o-mini")

content='Hello! How can I assist you today?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 8, 'total_tokens': 17, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0705bf87c0', 'finish_reason': 'stop', 'logprobs': None} id='run-54c474d4-1fd6-4da1-8e8c-a4b5396e9735-0' usage_metadata={'input_tokens': 8, 'output_tokens': 9, 'total_tokens': 17, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


# 3. 문서 로드

In [ ]:
from langchain.document_loaders import PyPDFLoader

# PDF 파일 로드. 파일의 경로 입력
loader = PyPDFLoader("/content/[2024 한권으로 OK 주식과 세금].pdf")

# 페이지 별 문서 로드
docs = loader.load()

print(docs)
print(len(docs))

[Document(metadata={'source': '/content/[2024 한권으로 OK 주식과 세금].pdf', 'page': 0}, page_content=''), Document(metadata={'source': '/content/[2024 한권으로 OK 주식과 세금].pdf', 'page': 1}, page_content=''), Document(metadata={'source': '/content/[2024 한권으로 OK 주식과 세금].pdf', 'page': 2}, page_content='지난해 말 국내 상장법인 주식을 보유한 개인 투자자가 1,400만명을 넘어서는 등  \n국민들의 주식시장에 대한 관심이 크게 증가하였습니다.\n최근 일반 국민들의 주식투자에 대한 관심이 크게 증가했음에도 불구하고, 주식 투자  \n관련 세금문제 등 궁금한 사항에 대하여 도움을 줄 수 있는 안내책자 등은 시중에서  \n쉽게 찾아보기 어려운 게 현실입니다.\n이에 국세청에서는 주식 관련 각종 세금에 대한 납세자들의 이해를 높이고 납세의무  \n이행에 도움이 될 수 있도록 「주식과 세금」 책자를 처음으로 제작·발간하게 되었습니다.\n이번에 새롭게 출간하는 ‘주식과 세금’ 책자는 주식거래의 기초상식과 주식의 취득  \n부터 보유 및 처분시까지 단계별 세금문제를 총 76개의 문답형식으로 구성하는 한편, \n인포그래픽 등을 적극 활용하여 가독성을 제고하였으며, 구체적인 절세 꿀팁 및 자주 \n발생하는 실수 사례 등을 추가하여 활용성도 강화하였습니다.\n모쪼록, 이 책자가 주식등 관련 납세자들의 성실한 납세의무 이행에 기여할 수 있기를\n기대합니다.\n2024.  5\n국세청 자산과세국장\n머리말'), Document(metadata={'source': '/content/[2024 한권으로 OK 주식과 세금].pdf', 'page': 3}, page_content='본 책자에

In [ ]:
print(docs[50])

page_content='39
양도소득세 과세대상 출자지분은 주로 인적회사의 지분을 의미하며, 여기서 인적회사란 
사원 상호간의 결합 관계가 깊고 회사의 활동이 사원들의 인적 조건에 따라 이루어지는 
회사로 사원들이 공동 출자하고 경영에도 참여하며 책임도 함께 지는  
회사를 말합니다. 합명회사, 합자회사가 대표적인 인적회사에  
해당합니다.
참고참고
양도소득세 과세대상 '주식등'이란 주식 또는 출자지분을 말하며, 신주인수권과 일정한 
증권예탁증권(DR)이 포함됩니다.
국내주식등 및 국내에 주소 또는 거소를 둔 거주자가 양도한 국외주식등의 양도로 발생하는 
소득에 대하여 양도소득세가 과세됩니다.
가. 양도소득세 과세대상 주식등의 종류 및 구분
조금 더 알아보기
출자지분의 개념
양도소득세 과세대상인 '주식등'이란 주식 또는 출자지분을 말하며,  
신주인수권과 일정한 증권예탁증권(DR)이 포함됩니다.
아울러, 국외주식도 양도소득세 과세대상에 해당합니다.
14    양도소득세 과세대상인
          '주식등'의 범위는 어떻게 되나요?
제 2 절 주식등 양도소득 과세대상' metadata={'source': '/content/[2024 한권으로 OK 주식과 세금].pdf', 'page': 50}


In [ ]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=100,
    chunk_overlap=10,
    length_function=len,
    is_separator_regex=False,
)

splits = text_splitter.split_documents(docs)
print(splits)
print(len(splits))
print(splits[0])

[Document(metadata={'source': '/content/[2024 한권으로 OK 주식과 세금].pdf', 'page': 2}, page_content='지난해 말 국내 상장법인 주식을 보유한 개인 투자자가 1,400만명을 넘어서는 등  \n국민들의 주식시장에 대한 관심이 크게 증가하였습니다.\n최근 일반 국민들의 주식투자에 대한 관심이 크게 증가했음에도 불구하고, 주식 투자  \n관련 세금문제 등 궁금한 사항에 대하여 도움을 줄 수 있는 안내책자 등은 시중에서  \n쉽게 찾아보기 어려운 게 현실입니다.\n이에 국세청에서는 주식 관련 각종 세금에 대한 납세자들의 이해를 높이고 납세의무  \n이행에 도움이 될 수 있도록 「주식과 세금」 책자를 처음으로 제작·발간하게 되었습니다.\n이번에 새롭게 출간하는 ‘주식과 세금’ 책자는 주식거래의 기초상식과 주식의 취득  \n부터 보유 및 처분시까지 단계별 세금문제를 총 76개의 문답형식으로 구성하는 한편, \n인포그래픽 등을 적극 활용하여 가독성을 제고하였으며, 구체적인 절세 꿀팁 및 자주 \n발생하는 실수 사례 등을 추가하여 활용성도 강화하였습니다.\n모쪼록, 이 책자가 주식등 관련 납세자들의 성실한 납세의무 이행에 기여할 수 있기를\n기대합니다.\n2024.  5\n국세청 자산과세국장\n머리말'), Document(metadata={'source': '/content/[2024 한권으로 OK 주식과 세금].pdf', 'page': 3}, page_content='본 책자에 수록된 내용은 세법에서 규정하고 있는 내용을 알기 쉽게 요약하여 서술한\n것으로 모든 법령규정을 담고 있지는 않습니다.\n또한, 법령이 개정되거나 법령의 해석이 변경되어 본 책자의 내용과 다른 경우가 발생\n할 수 있으므로 실제 사안에 적용하는 경우 반드시 관련 법령과 해석 등을 충분히 확인\n하시기 바랍니다.\n본 책자는 발간일 현재 개정된 법령 등을 기준으로 작성되었습니다. 다만, 시행

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

recursive_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=10,
    length_function=len,
    is_separator_regex=False,
)

splits = recursive_text_splitter.split_documents(docs)
print(splits)
print(len(splits))
print(splits[0])

[Document(metadata={'source': '/content/[2024 한권으로 OK 주식과 세금].pdf', 'page': 2}, page_content='지난해 말 국내 상장법인 주식을 보유한 개인 투자자가 1,400만명을 넘어서는 등  \n국민들의 주식시장에 대한 관심이 크게 증가하였습니다.'), Document(metadata={'source': '/content/[2024 한권으로 OK 주식과 세금].pdf', 'page': 2}, page_content='최근 일반 국민들의 주식투자에 대한 관심이 크게 증가했음에도 불구하고, 주식 투자  \n관련 세금문제 등 궁금한 사항에 대하여 도움을 줄 수 있는 안내책자 등은 시중에서'), Document(metadata={'source': '/content/[2024 한권으로 OK 주식과 세금].pdf', 'page': 2}, page_content='쉽게 찾아보기 어려운 게 현실입니다.\n이에 국세청에서는 주식 관련 각종 세금에 대한 납세자들의 이해를 높이고 납세의무'), Document(metadata={'source': '/content/[2024 한권으로 OK 주식과 세금].pdf', 'page': 2}, page_content='이행에 도움이 될 수 있도록 「주식과 세금」 책자를 처음으로 제작·발간하게 되었습니다.\n이번에 새롭게 출간하는 ‘주식과 세금’ 책자는 주식거래의 기초상식과 주식의 취득'), Document(metadata={'source': '/content/[2024 한권으로 OK 주식과 세금].pdf', 'page': 2}, page_content='부터 보유 및 처분시까지 단계별 세금문제를 총 76개의 문답형식으로 구성하는 한편, \n인포그래픽 등을 적극 활용하여 가독성을 제고하였으며, 구체적인 절세 꿀팁 및 자주'), Document(metadata={'sou

In [ ]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002") # 토큰화된 문서를 모델에 입력하여 임베딩 벡터를 생성하고, 이를 평균하여 전체 문서의 벡터를 생성

In [ ]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)

# 7. Retriever 객체 생성


In [ ]:
from langchain.vectorstores.base import VectorStore

retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 10})

# 8. 프롬프트 템플릿 정의

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# 9. RAG 체인 구성

In [ ]:
from langchain.chains import LLMChain

# 디버깅을 위해 만든 클래스 (신경쓰지 않으셔도 됩니다.)
class SimplePassThrough:
    def invoke(self, inputs, **kwargs):
        return inputs

# 프롬프트 클래스
class ContextToPrompt:
    def __init__(self, prompt_template):
        self.prompt_template = prompt_template

    def invoke(self, inputs):
        # response_docs 내용을 trim해줌 (가독성을 높여줌)
        if isinstance(inputs, list): # inputs가 list인 경우. 즉 여러개의 문서들이 검색되어 리스트로 전달된 경우
            context_text = "\n".join([doc.page_content for doc in inputs]) # \n을 구분자로 넣어서 한 문자열로 합쳐줌
        else:
            context_text = inputs # 리스트가 아닌경우는 그냥 리턴해줌

        # 프롬프트
        formatted_prompt = self.prompt_template.format_messages( # 템플릿의 변수에 삽입해줌
            context=context_text, # {context} 변수에 context_text, 즉 검색된 문서 내용을 삽입함
            question=inputs.get("question", "")
        )
        return formatted_prompt

# Retriever 클래스
class RetrieverWrapper:
    def __init__(self, retriever):
        self.retriever = retriever

    def invoke(self, inputs):
        if isinstance(inputs, dict): # inputs가 딕셔너리일경우, question 키의 값을 검색 쿼리로 사용
            query = inputs.get("question", "")
        else: # 질문이 문자열로 주어지면, 그대로 검색 쿼리로 사용
            query = inputs
        response_docs = self.retriever.get_relevant_documents(query) # 검색을 수행하고 검색 결과를 response_docs에 저장
        return response_docs

# 10. 챗봇 구동 확인

In [ ]:
# 외부 프롬프트 파일을 불러오기
def load_prompt_from_file(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        return file.read()

In [ ]:
import time

# 경로 설정
prompt_dir = "./Prompts/"
result_dir = "./Results/"

# 디렉토리 생성 (필요시)
os.makedirs(result_dir, exist_ok=True)

# 챗봇 구동
while True:
    # 프롬프트 파일 선택
    prompt_files = [f for f in os.listdir(prompt_dir) if f.endswith(".txt")]
    if not prompt_files:
        print("프롬프트 파일이 없습니다. Prompts/ 디렉토리를 확인하세요.")
        break

    print("\n사용 가능한 프롬프트 파일:")
    for idx, prompt_file in enumerate(prompt_files, 1):
        print(f"{idx}. {prompt_file}")

    file_choice = int(input("\n사용할 프롬프트 파일 번호를 선택하세요: ")) - 1
    if file_choice < 0 or file_choice >= len(prompt_files):
        print("잘못된 선택입니다.")
        continue

    prompt_path = os.path.join(prompt_dir, prompt_files[file_choice])
    contextual_prompt = load_prompt_from_file(prompt_path)
    print(f"\n선택된 프롬프트 파일: {prompt_files[file_choice]}")

    # RAG 체인 설정
    rag_chain_debug = {
        "context": RetrieverWrapper(retriever),
        "prompt": ContextToPrompt(contextual_prompt),
        "llm": model
    }

    print("\n========================")

    # 0. 질문을 받아서 query에 저장함
    query = input("질문을 입력하세요 : ")

    # 1. 리트리버로 question에 대한 검색 결과를 response_docs에 저장함
    response_docs = rag_chain_debug["context"].invoke({"question": query})

    # 2. 프롬프트에 질문과 response_docs를 넣어줌
    prompt_messages = rag_chain_debug["prompt"].invoke({
        "context": response_docs,
        "question": query
    })

    # 3. 완성된 프롬프트를 LLM에 넣어줌
    response = rag_chain_debug["llm"].invoke(prompt_messages)

    print("\n답변:")
    print(response.content)

    # 4. 결과 외부 Result 폴더에 저장
    timestamp = int(time.time()) # 타임스탬프 기록
    result_filename = f"{prompt_files[file_choice].replace('.txt', '')}_result_{timestamp}.txt"
    result_path = os.path.join(result_dir, result_filename)
    with open(result_path, "w", encoding="utf-8") as result_file:
      result_file.write(response.content)


질문을 입력하세요 : 시간외바스켓매매가 뭐야?

답변:
시간외바스켓매매는 투자자에게 추가적인 매매거래 기회를 제공하기 위한 방법 중 하나로, 시간외 시장에서 여러 종목의 주식을 동시에 대량으로 매매하는 거래 방식입니다. 이 매매는 정규시장 외의 시간에 이루어지며, 매도·매수 당사자 간 합의한 가격으로 체결됩니다.


KeyboardInterrupt: Interrupted by user